---
# Guía Perceptrón

En estea guía realizaremos las siguientes actividades:
- Repasar conceptos fundamentales del perceptrón
- Resolver desafíos


### El Perceptrón

Un perceptrón es un algoritmo de aprendizaje supervisado en el campo del aprendizaje automático y la inteligencia artificial. 
Se utiliza para la clasificación de datos y forma la base de las redes neuronales artificiales. 
Fue propuesto por Frank Rosenblatt en 1957 y se considera una de las formas más simples de una neurona artificial.

En términos simples, un perceptrón toma un conjunto de entradas, realiza una combinación lineal de estas entradas multiplicadas por los pesos correspondientes, 
y luego aplica una función de activación para producir una salida. 
Esta salida puede ser binaria (0 o 1) o puede ser una salida continua, dependiendo de la función de activación utilizada.

La función de activación típicamente utilizada en un perceptrón es una función escalón (step function), que devuelve 1 si la suma ponderada de las entradas es mayor 
o igual a un cierto umbral, y 0 en caso contrario. Esto significa que un perceptrón puede aprender a clasificar datos en dos categorías, 
separando los puntos en un espacio dimensional en dos regiones con una línea (o hiperplano en dimensiones superiores).



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


### Implementación

El siguiente código define una clase Perceptrón con métodos para la inicialización, entrenamiento y predicción de valores.

En el método `__init__`, inicializamos los atributos del perceptrón. `input_size` representa el número de características (o entradas) que tiene cada instancia de datos. `learning_rate` es la tasa de aprendizaje del perceptrón, que determina cuánto se ajustan los pesos durante el entrenamiento. `epochs` es el número de iteraciones que realizaremos sobre el conjunto de datos durante el entrenamiento.


El método `predict` toma las entradas (inputs) y calcula la suma ponderada de las entradas multiplicadas por los pesos, más un término de sesgo (`self.weights[0]`). Luego, aplica una función de activación, que en este caso es una función escalón, y devuelve la salida (0 o 1).

El método `train` realiza el entrenamiento del perceptrón. Itera sobre el conjunto de datos `epochs` veces. En cada iteración, itera sobre cada ejemplo de entrenamiento `(inputs, label)` en `training_inputs` y `labels`. Para cada ejemplo, realiza una predicción utilizando el método `activation`. Luego, ajusta los pesos del perceptrón según el error entre la predicción y la etiqueta verdadera, multiplicado por la tasa de aprendizaje y las entradas. Esto se hace utilizando la regla de aprendizaje del perceptrón.

In [ ]:
class Perceptron:
    def __init__(self, input_size, learning_rate=0.01, epochs=100):
        self.weights = np.zeros(input_size + 1)
        self.learning_rate = learning_rate
        self.epochs = epochs

    def train(self, training_inputs, labels):
        for _ in range(self.epochs):
            for inputs, label in zip(training_inputs, labels):
                prediction = self.activation(inputs)
                self.weights[1:] += self.learning_rate * (label - prediction) * inputs
                self.weights[0] += self.learning_rate * (label - prediction)

    def activation(self, input):
        summation = np.dot(input, self.weights[1:]) + self.weights[0]
        return 1 if summation > 0 else 0

    def predict(self, input_array):
        return np.array([self.activation(x) for x in input_array])



Hay que recalcar que esta implementación está hecha para resolver problemas de **clasificación binaria**.

**Carga del set de datos**

Ahora utilizaremos el set de datos de notas de estudiantes versus horas de estudio. Recuerde que la nota de aprobación es 55, por lo tanto, debe agregar una columna nueva con valor 0 si no aprueba, y con valor 1 si aprueba. Esa columna correspondería a la variable objetivo con la cual vamos a etiquetar los datos.

In [ ]:
df = pd.read_csv('student_scores.csv')
df.head(2)

In [ ]:
# haga el wrangling para agregar la variable objetivo al set de datos
df['aprueba'] = (df['Scores'] >= 55).astype(int)
df.head(2)

**Definición del modelo**

Defina la matriz de features `X` y el vector de etiquetas `y`. Recuerde que estas variables deben ser arreglos numpy para que la clase `Perceptron` pueda tratarlos.

In [ ]:
# en este caso, trabajaremos con arreglos numpy, por eso utilizamos .value
X = df[['Hours']].values
y = df['aprueba'].values

**Validación cruzada**

Divida el set de entrenamiento en training y test set.

In [ ]:
# importar funcion para division de datos
from sklearn.model_selection import train_test_split

In [ ]:
# divida el set de entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Entrenamiento del Perceptrón**

Realice el entrenamiento del perceptrón con valores por defecto para learning rate y epochs. Note que en este caso, la matriz de features tiene solamente una entrada, por lo cual `input_size` tiene valor 1.

In [ ]:
# Crear y entrenar el perceptrón
perceptron = Perceptron(input_size=1)
perceptron.train(X_train, y_train)

In [ ]:
# con el modelo recién ajustado, haga una predicción para un estudiante que dedica 5 horas de estudio
# ¿aprueba la asignatura?
perceptron.predict([[5]])



**Evaluación**

Aplique las métricas de evaluación al modelo entrenado.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
y_pred  = perceptron.predict(X_test)

In [ ]:
# accuracy
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
# matriz de confusión
cm = confusion_matrix(y_test, y_pred)
cm

Como se puede observar, hubo un alto accuracy en este modelo.

### Perceptrón Titanic

Ahora veremos si podemos aplicar este perceptrón para resolver el problema del Titanic.

In [ ]:
df = pd.read_csv('titanic.csv')
df.head(2)

**Limpieza de datos**

Para simpliplicar la operatoria, simplemente eliminaremos los registros que tienen valores nulos en las columnas en donde realizaremos el modelo.

In [ ]:
# realizar tratamiento de valores nulos
df = df.dropna(subset=['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked'])
df = df.reset_index(drop=True)
df.head(2)

**Definir el modelo**

Como es habitual, definimos la matriz de features X y el vector de resultados y.

In [ ]:
X = df[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']]
y = df['Survived'].values

**Preprocesamiento**

En este problema, debemos realizar el siguiente preprocesamiento:
- Binarizar columnas categóricas
- Escalar los datos


In [ ]:
# binarización
X = pd.get_dummies(X, columns=['Sex', 'Embarked'], drop_first=True)
X.head(2)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# escalamiento
X_scaled[:2]

**Validación cruzada**

Aplique división del set de datos para entrenar y testear el modelo.

In [ ]:
# division del set de datos (recuerde que debe hacerlo con los datos escalados y binarizados)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

**Entrenamiento**


In [ ]:
# Crear y entrenar el perceptrón (recuerde que la matriz de features tiene dimensiones diferentes que en el ejemplo anterior)
perceptron_titanic = Perceptron(input_size=X_train.shape[1], learning_rate=0.01, epochs=1000)
perceptron_titanic.train(X_train, y_train)

**Evaluación del modelo**

In [ ]:
# predicciones sobre el set de test
y_pred_titanic = perceptron_titanic.predict(X_test)

In [ ]:
# accuracy
accuracy_titanic = accuracy_score(y_test, y_pred_titanic)
accuracy_titanic

In [ ]:
# matriz de confusion
cm_titanic = confusion_matrix(y_test, y_pred_titanic)
cm_titanic

**Afinamiento del algoritmo**

Realice una optimización de los hiperparámetros del algoritmo. Pruebe con varias combinaciones de learning_rate y epochs.

In [ ]:
# búsqueda simple de hiperparámetros para el perceptrón del Titanic
learning_rates = [0.001, 0.01, 0.05, 0.1]
epoch_list = [100, 500, 1000, 2000]

results = []

for lr in learning_rates:
    for n_epochs in epoch_list:
        model = Perceptron(input_size=X_train.shape[1], learning_rate=lr, epochs=n_epochs)
        model.train(X_train, y_train)
        y_pred_grid = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred_grid)
        results.append({'learning_rate': lr, 'epochs': n_epochs, 'accuracy': acc})

In [ ]:
# visualizar resultados de la búsqueda de hiperparámetros
results_df = pd.DataFrame(results)
results_df.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

In [ ]:
# mejor combinación de hiperparámetros encontrada
best = max(results, key=lambda x: x['accuracy'])
best

#### Conclusiones

¿Qué conclusiones puede elaborar después de esta experiencia?

In [ ]:
# conclusiones acá
conclusiones = """\
En este ejercicio observamos que:
- El perceptrón funciona bien para problemas de clasificación binaria linealmente separables.
- En el caso de student_scores, la relación horas de estudio / aprobación es casi lineal y el modelo logra alta exactitud.
- En Titanic, el problema es más complejo y depende de varias variables; el perceptrón puro puede tener un desempeño limitado frente a modelos más sofisticados.
- El preprocesamiento (binarización y escalamiento) es clave para obtener buenos resultados.
- El ajuste de hiperparámetros (learning_rate y epochs) puede mejorar el desempeño, pero con rendimientos decrecientes.
"""
print(conclusiones)

---